# Notebook relating pictures with new backgrounds to their ground truth mask

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import cv2
from IPython.display import clear_output
import sys 
sys.path.append('../../src/')
from data_process import label_2_colour

## Read all the images with new backgrounds from its folder

In [ ]:
new_bk_path = "../../data/new_background/"
new_bk_names = glob.glob(new_bk_path + "*.png")
new_bk_names.sort()
len(new_bk_names)

In [2]:
def combine_image_and_labels(img_path, label_path):
    """Function combining the new background image with the original label

    Args:
        img_path (str): path to the new background image
        label_path (str): path to the original label

    Returns:
        ndarray: array with the new background and the original label
    """
    org_arr = np.load(label_path)
    aug_arr = cv2.imread(img_path)
    aug_arr = cv2.cvtColor(aug_arr, cv2.COLOR_BGR2RGB) # cv2 reads bgr default
    new_arr = np.zeros_like(org_arr)
    new_arr[:,:,0:3] = aug_arr[:,:,0:3] # take the rgb info from the new image
    new_arr[:,:,3] = org_arr[:,:,3]     # take the label info from the original image

    return new_arr

def find_corresponding_label(img_path):
    """Function to find the corresponding label for the image

    Args:
        img_path (str): path to the image
        all_labels (list): list of all the labels

    Returns:
        str: path to the corresponding label
    """
    img_name = img_path.split("/")[-1]
    label_name = img_name.split(".")[0][:-3] + ".npy"   # Removes the _bk from name and adds .npy
    label_path = os.path.join("../data/carseg_data/augmented_arrays", label_name)
    
    return label_path

## Relate the new background image to its corresponding label

In [ ]:
org_names = glob.glob("../../data/carseg_data/augmented_arrays/*.npy")
org_names.sort()

# Loops through all the images and finds its corresponding label, before saving
for img in new_bk_names:
    name = img.split("/")[-1].split(".")[0] 
    lab = find_corresponding_label(img)
    new_arr = combine_image_and_labels(img, lab)
    np.save(os.path.join("../data/carseg_data/new_background_arrays/", "{}.npy".format(name)), new_arr)


In [ ]:
# Visualise the new images with new background side by side with labels
import time
new_arrs = glob.glob("../../data/new_dataset/*.npy")[:100]
new_arrs.sort()
new_arrs = np.array([np.load(arr) for arr in new_arrs])
for i in range(len(new_arrs)):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.imshow(new_arrs[i][:,:,0:3])
    plt.subplot(1,2,2)
    plt.imshow(label_2_colour(new_arrs[i][:,:,3]))
    plt.show()
    time.sleep(0.001)
    clear_output(wait=True)